# Tensorflow ImageNet Pixel Threshold demo

>⚠️ **Warning:** This demo assumes that you have access to an on-prem deployment of Dioptra that provides a copy of the ImageNet dataset and a CUDA-compatible GPU.
> This demo cannot be run on a typical personal computer.

This notebook contains a demonstration of how to use Dioptra to run experiments that investigate the effects of the pixel threshold attack when launched on a neural network model trained on the ImageNet dataset.

## Setup

**Note:** This demo is specifically for the NCCoE DGX Workstation with hostname `dgx-station-2`.

Port forwarding is required in order to run this demo.
The recommended port mapping is as follows:

- Map `*:20080` on laptop to `localhost:30080` on `dgx-station-2`
- Map `*:25000` on laptop to `localhost:35000` on `dgx-station-2`

A sample SSH config file that enables the above port forwarding is provided below,

> ⚠️ **Edits required**: replace `username` with your assigned username _on the NCCoE virtual machines_!

```conf
# vm hostname: jumphost001
Host nccoe-jumphost001
    Hostname 10.33.53.98
    User username  # Change to your assigned username on the NCCoE virtual machines!
    Port 54131
    IdentityFile %d/.ssh/nccoe-vm

# vm hostname: dgx-station-2
Host nccoe-k8s-gpu002
    Hostname 192.168.1.28
    User username  # Change to your assigned username on the NCCoE virtual machines!
    Port 22
    IdentityFile %d/.ssh/nccoe-vm
    ProxyJump nccoe-jumphost001
    LocalForward *:20080 localhost:30080
    LocalForward *:25000 localhost:35000
```

Now, connect to the NCCoE VPN and SSH into the DGX Workstation,

```bash
ssh nccoe-k8s-gpu002
```

Next, we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

> ⚠️ **Edits possibly required**: update the value of the `HOST_DOCKER_INTERNAL` variable

If this notebook is being served to you via Docker (i.e. you ran `make jupyter` to launch this notebook), **then you may need to change the value assigned to the variable `HOST_DOCKER_INTERNAL`** to make the port forwarding you configured in the previous step accessible within the container.
The value you need to assign to the variable to depends on your host device's operating system:

- **Case 1: Host operating system is Windows 10 or MacOS**
  - Set `HOST_DOCKER_INTERNAL = "host.docker.internal"`. This is the default setting.
- **Case 2: Host operating system is Linux**
  - Run either `ip address` or `ifconfig` to print a list of the available network interfaces on your host device
  - Locate the `docker0` interface and take note of the associated IP address (this is commonly set to `172.17.0.1`)
  - Set `HOST_DOCKER_INTERNAL` equal to the IP address for the `docker0` interface. So, if the IP address was `172.17.0.1`, then you would set `HOST_DOCKER_INTERNAL = "172.17.0.1"`
  
If you started your Jupyter Lab instance from a conda environment, then you do not need to change anything.
The code below uses an environment variable to check whether this notebook is being served via the `jupyter` service, and if that variable isn't found, then the connection address reverts to `localhost` and ignores the `HOST_DOCKER_INTERNAL` variable.

In [ ]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Dioptra API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the DIOPTRA_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = "jtsexton_imagenet_pixel_threshold"

# Path to dataset
data_path_imagenet = "/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC"

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make workflows

To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `DIOPTRA_RESTAPI_URI` to figure out how to connect to the Dioptra RESTful API,

In [ ]:
restapi_client = utils.DioptraClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=experiment)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=experiment)

response_experiment

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Lab API and returns a list of active queues.

In [ ]:
restapi_client.list_queues()

Next, we need to create our model.
In this example, we will be using an existing ImageNet classifier, and not training a new one. So, we use the ```init_model``` entry point to simply initialize an existing model.

In [ ]:
response_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="init_model",
    entry_point_kwargs=" ".join([
        f"-P data_dir={data_path_imagenet}/val-sorted-5000",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

pprint.pprint(response_train)

Now that we have an ImageNet model, we can run the Pixel Threshold attack on it.
The Pixel Threshold attack attempts to change a limited number of pixels in a test image in an attempt to get it misclassified.
It has two main arguments:

| parameter | data type | description |
| --- | --- | --- |
| th | int | The maximum number of pixels it is allowed to change |
| `es` | int | If 0, then use the CMA-ES strategy, or if 1, use the DE strategy for evolution |

In [ ]:
def mlflow_run_id_is_not_known(response_pt):
    return response_pt["mlflowRunId"] is None and response_pt["status"] not in [
        "failed",
        "finished",
    ]

response_pt = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="pt",
    entry_point_kwargs=" ".join(
        [
            f"-P model=keras-model-imagenet-resnet50/6",
            "-P model_architecture=alex_net",
            f"-P data_dir={data_path_imagenet}",
            "-P batch_size=32",
            "-P th=1",
            "-P es=1",
        ]
    ),
    queue="tensorflow_gpu",
    timeout="1h"
)

pprint.pprint(response_pt)